# 多用public属性，少用private属性

在Python类，其属性可见度只有两种，也就是public和private

In [1]:
import logging
from pprint import pprint

In [2]:
class MyObject(object):
    def __init__(self):
        self.public_field = 5
        self.__private_field = 10
    
    def get_private_field(self):
        return self.__private_field

In [3]:
foo = MyObject()
assert foo.public_field == 5

In [4]:
assert foo.get_private_field() == 10

由于类级别的方法仍然声明在本类的class代码块内，所以这些方法也能够访问private属性。

In [5]:
class MyOtherObject(object):
    def __init__(self):
        self.__private_field = 71
        
    @classmethod
    def get_private_field_of_instance(cls, instance):
        return instance.__private_field

In [6]:
bar = MyOtherObject()
assert MyOtherObject.get_private_field_of_instance(bar) == 71

子类无法访问父类的private字段。

In [7]:
class MyParentObject(object):
    def __init__(self):
        self.__private_field = 71
    
class MyChildObject(MyParentObject):
    def get_private_field(self):
        return self.__private_field

In [8]:
try:
    baz = MyChildObject()
    baz.get_private_field()
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-8-e6f935b1ea27>", line 3, in <module>
    baz.get_private_field()
  File "<ipython-input-7-1a95091faa00>", line 7, in get_private_field
    return self.__private_field
AttributeError: 'MyChildObject' object has no attribute '_MyChildObject__private_field'


In [9]:
assert baz._MyParentObject__private_field == 71

In [10]:
print(baz.__dict__)

{'_MyParentObject__private_field': 71}


**Python为什么不从语法上严格保证private字段的私密性呢？用最简单的话来说，就是：We are all consenting adults here（我们都是成年人）**

**习惯性命名：**以单个下划线开头的字段，应该视为protected字段，本类之外的代码在使用这种字段的时候要多加小心。

In [11]:
class MyClass(object):
    def __init__(self, value):
        self.__value = value
    
    def get_value(self):
        return str(self.__value)

In [12]:
foo = MyClass(5)
assert foo.get_value() == '5'

**问题：**假如超类使用了private属性，那么子类在覆写或扩展的时候，就会遇到麻烦和错误。继承该类的那些子类，在万不得已的时候，仍然要去访问private字段。

In [13]:
class MyIntegerSubclass(MyClass):
    def get_value(self):
        return int(self._MyClass__value)

In [14]:
foo = MyIntegerSubclass(5)
assert foo.get_value() == 5

**改进办法：**宁可让子类更多地去访问超类的protected属性，也别把这些属性设成private。

In [15]:
class MyBaseClass(object):
    def __init__(self, value):
        self.__value = value

    def get_value(self):
        return self.__value

In [16]:
class MyClass(MyBaseClass):
    def get_value(self):
        return str(super().get_value())

In [17]:
class MyIntegerSubclass(MyClass):
    def get_value(self):
        return int(self._MyClass__value)

In [18]:
try:
    foo = MyIntegerSubclass(5)
    foo.get_value()
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-18-dd646a972bd6>", line 3, in <module>
    foo.get_value()
  File "<ipython-input-17-e1378a58f70f>", line 3, in get_value
    return int(self._MyClass__value)
AttributeError: 'MyIntegerSubclass' object has no attribute '_MyClass__value'


在文档中说明每个protected字段的含义，解释哪些字段可供子类使用的内部API、哪些字段是完全不应使用的数据。

In [19]:
class MyClass(object):
    def __init__(self, value):
        # This stores the user-supplied value for the object.
        # It should be coercible to a string. Once assigned for
        # the object it should be treated as immutable.
        self._value = value

    def get_value(self):
        return str(self._value)

**只有一种情况可以合理使用private属性，那就是用它来避免子类的属性名与超类相冲突。**

In [20]:
class ApiClass(object):
    def __init__(self):
        self._value = 5

    def get(self):
        return self._value

In [21]:
class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello'  # Conflicts

In [22]:
a = Child()
print(a.get(), 'and', a._value, 'should be different')

hello and hello should be different


**解决方法：**可以在超类中使用private属性，以确保子类的属性名不会与之重复。

In [23]:
class ApiClass(object):
    def __init__(self):
        self.__value = 5

    def get(self):
        return self.__value

In [24]:
class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello'  # OK!

In [25]:
a = Child()
print(a.get(), 'and', a._value, 'are different')

5 and hello are different
